Importation des library

In [30]:
import numpy as np
import pandas as pd
import vectorbt as vbt

symbol = ['ETHUSDT']
time = "1h"
chemin = r'C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1h\ETHUSDT\ETHUSDT_1years_2024-12-22_TO_2025-12-26_1h_data'
data = pd.read_parquet(chemin)


# =============================
# PARAMÈTRES
# =============================
rsi_length = 14
lookback_left = 3
lookback_right = 3

min_div_bars = 10
max_div_bars = 200

use_close_confirmation = True
use_volume_confirmation = False
use_trend_filter = True

ema_length = 200

stop_loss_pct = 3.0
take_profit_pct = 6.0
trailing_stop = True

use_hidden_div = True
use_regular_div = True

rsi_zone = "Both"

# =============================
# DONNÉES
# =============================
df = data.copy()
df.columns = df.columns.str.lower()

close = df['close']
high = df['high']
low = df['low']
volume = df['volume']

# =============================
# INDICATEURS
# =============================
rsi = vbt.RSI.run(close, window=rsi_length).rsi
ema = vbt.MA.run(close, window=ema_length).ma
ema_shifted = ema.shift(1)

volume_avg = volume.rolling(20).mean()
volume_high = volume > volume_avg * 1.3

entry_delay = 1 if use_close_confirmation else 0

# =============================
# SIGNALS
# =============================
long_entries = np.zeros(len(df), dtype=bool)
short_entries = np.zeros(len(df), dtype=bool)

price_highs, rsi_highs, bar_highs = [], [], []
price_lows, rsi_lows, bar_lows = [], [], []

# =============================
# BOUCLE PRINCIPALE
# =============================
for i in range(len(df)):

    # ========= PIVOTS =========
    if i >= lookback_left and i + lookback_right < len(df):

        # ---- HIGH PIVOT ----
        if high[i] == high[i-lookback_left:i+lookback_right+1].max():
            pivot_bar = i - lookback_right
            price_highs.insert(0, high[i])
            rsi_highs.insert(0, rsi[i])
            bar_highs.insert(0, pivot_bar)

        # ---- LOW PIVOT ----
        if low[i] == low[i-lookback_left:i+lookback_right+1].min():
            pivot_bar = i - lookback_right
            price_lows.insert(0, low[i])
            rsi_lows.insert(0, rsi[i])
            bar_lows.insert(0, pivot_bar)

        price_highs = price_highs[:10]
        rsi_highs = rsi_highs[:10]
        bar_highs = bar_highs[:10]

        price_lows = price_lows[:10]
        rsi_lows = rsi_lows[:10]
        bar_lows = bar_lows[:10]

    bullish = bearish = False
    hidden_bullish = hidden_bearish = False
    signal_bar = None

    # ===== REGULAR =====
    if use_regular_div and len(price_highs) >= 2:
        bars = abs(bar_highs[0] - bar_highs[1])
        if price_highs[0] > price_highs[1] and rsi_highs[0] < rsi_highs[1] and min_div_bars <= bars <= max_div_bars:
            bearish = True
            signal_bar = bar_highs[0]

    if use_regular_div and len(price_lows) >= 2:
        bars = abs(bar_lows[0] - bar_lows[1])
        if price_lows[0] < price_lows[1] and rsi_lows[0] > rsi_lows[1] and min_div_bars <= bars <= max_div_bars:
            bullish = True
            signal_bar = bar_lows[0]

    # ===== HIDDEN =====
    if use_hidden_div and len(price_highs) >= 2:
        bars = abs(bar_highs[0] - bar_highs[1])
        if price_highs[0] < price_highs[1] and rsi_highs[0] > rsi_highs[1] and min_div_bars <= bars <= max_div_bars:
            hidden_bearish = True
            signal_bar = bar_highs[0]

    if use_hidden_div and len(price_lows) >= 2:
        bars = abs(bar_lows[0] - bar_lows[1])
        if price_lows[0] > price_lows[1] and rsi_lows[0] < rsi_lows[1] and min_div_bars <= bars <= max_div_bars:
            hidden_bullish = True
            signal_bar = bar_lows[0]

    # ===== CONDITIONS =====
    long_cond = False
    short_cond = False

    if bullish and rsi_zone in ["Both", "Oversold Only"]:
        long_cond = True
    if hidden_bullish and rsi_zone in ["Both", "Overbought Only"]:
        long_cond = True

    if bearish and rsi_zone in ["Both", "Overbought Only"]:
        short_cond = True
    if hidden_bearish and rsi_zone in ["Both", "Oversold Only"]:
        short_cond = True

    # ===== ENTRÉE (>= COMME PINE) =====
    if signal_bar is not None and i >= signal_bar + entry_delay:

        if use_trend_filter:
            long_cond &= close[i] > ema_shifted[i]
            short_cond &= close[i] < ema_shifted[i]

        if use_volume_confirmation:
            long_cond &= volume_high[i]
            short_cond &= volume_high[i]

        long_entries[i] = long_cond
        short_entries[i] = short_cond

        
        
portfolio = vbt.Portfolio.from_signals(
    close,
    entries=long_entries,
    short_entries=short_entries,

    sl_stop=stop_loss_pct / 100,
    tp_stop=take_profit_pct / 100,
    sl_trail=0.01 if trailing_stop else None,

    init_cash=1000,
    freq='1h',
    direction='both',
    accumulate=False,
    fees=0.001,
)

portfolio.stats()
portfolio.plot().show()



C:\Users\gunsa\AppData\Local\Temp\ipykernel_17072\3557967659.py:77: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\gunsa\AppData\Local\Temp\ipykernel_17072\3557967659.py:84: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\gunsa\AppData\Local\Temp\ipykernel_17072\3557967659.py:86: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\gunsa\AppData\Local\Temp\ipykernel_17072\3557967659.py:87: FutureWarning:

Series.__g

In [31]:
portfolio.stats()

Start                         2024-12-22 00:00:00+01:00
End                           2025-12-26 16:00:00+01:00
Period                                369 days 17:00:00
Start Value                                      1000.0
End Value                                   1222.018312
Total Return [%]                              22.201831
Benchmark Return [%]                         -12.711895
Max Gross Exposure [%]                            100.0
Total Fees Paid                              252.556962
Max Drawdown [%]                              30.475177
Max Drawdown Duration                 158 days 18:00:00
Total Trades                                        118
Total Closed Trades                                 117
Total Open Trades                                     1
Open Trade PnL                               -25.993382
Win Rate [%]                                  35.897436
Best Trade [%]                                 8.632967
Worst Trade [%]                               -6